In [ ]:
from z3 import * 







https://www.cs.cornell.edu/courses/cs6120/2020fa/blog/alive/
https://blog.regehr.org/archives/1170

```
%a = sdiv %X, C 
%r = sub 0, %a 
  =>
%r = sdiv %X, -C
```
Overflow is also undefined behavior. That can really bite ya


```
Pre: C1 & C2 == 0 && MaskedValueIsZero(%V, ∼C1)
%t0 = or   %B,  %V
%t1 = and %t0,  C1
%t2 = and  %B,  C2
%R  = or  %t1, %t2
=>
%R  = and %t0, (C1 | C2)
```

Undefined behavior.
state -> option state. model 1. In the none case, nothing is returned

A different undefined / partial function model. 


## Pure Partiality Modelling
Division by zero is 

Which of these models is preferred:
- The one that does the least surprising things
- The one that allows the most desirable optimizations


Things that we think of as undefined even in a pure-ish context (we might mean by pure)
- Division by zero
- Non termination
- Ill typed combinations of terms. Occurs in categorical modelling, lowering stronger type systems (Coq) to weaker typed (z3).

THese are not orthogonal. A type `{x | x != 0}` might show up in coq and we just erase it, but now we need to deal with div-0

This is all a general quotienting/refinement phenomenon. We make a free mathemtical theory having more stuff than we watn and we try to cut away things via equations and preconditions of usage. It's often hard  (impossible?) to have a system that only expresses the things we want in a very deep sense by construction. I suppose this is the appeal of dependent types, but they are a distaster in their own way, and in another sense are still just this cutting phenomneon. The proof objligations of usage are the result of the non trivial quotienting / refinmenet.


The big idea of denotational semantics is to reify nontermination as a kind of partiality or special value. 

Consistent return values. In this example x/0 returns the same thing every time.

This is more like implementation defined behavior. The pure form of this is `impl_record -> value`. So the semantics of expressions that include divide by zero is `R -> R` whre we take in an implementation defined div 0.


We merely do not speak of what the function returns when it ecueives inappropriate stuff.
Undefinedness is self consistent in a single model, but can vary across models.

In [9]:
from z3 import *
x = Real("x")
prove(x/0 == x / 0)

solve(x / 0 <=  10) # here /0 
solve(x / 0 >  10)


proved
[x = 7719, /0 = [else -> 0]]
[x = 7719, /0 = [else -> 11]]


A different model is kind of turning x into a source of randomness. The randomness is samles at every use

We kind of want hilbert epslilon, but we don't even, because we want epsilon to choose something idfferent every time. I'm not sure if epsilon is required to always pick the same thing given the same predicate

We can also treat this relationally to compare with the previous method. If we constrain inputs to be equal of two freshly named programs, 
Optimizations are allowed that preserve correspondence or are allowed if the original program did not preserve correspondence.
This method is typical in information security modelling.

The "high" variables are the undefined ones and the "low" ones are the defined ones. We do not want high data leaking into low data. WHat is high vs low is intertwined with what happens, and not just particular variables. Things become high. Some I suppose maybe it's a 3 or 4 relation. 2 copies of original, 2 copies of modified.

We could also do a probablistic model of undefined behavior. It'd be surprising if this helped us. Maybe we could start to use pyro beysian update stuff for finding optimizations? Me dunno.


A lot of the endeavors  of programing language semantics is try and make sense of undefined beahvior. The most classic version of this is non termination, which has been with us since the dawn of theory of computation



This is kind of like unpacking an ADT into union struct form. We have tags that say which case we're in, but we keep around all the stuff.

In [ ]:
from dataclasses import dataclass

@dataclass 
class Var():
    is_undef : BoolRef
    is_poison : BoolRef
    val: ExprRef
    sort : SortRef

    def __init__(self):
        self.is_undef = FreshBool("undef")
        self.is_poison = FreshBool() 

    def read(self):
        # Hmm. Read needs to return poison, so it's a Maybe(Int). Maybe we should move this stuff into 
        return If(self.is_undef, FreshConst(self.sort),
               If(self.is_poison, 
                   
# us the CF/Max style output

    def __add__(x: Var, y : Var):
        return Var(
        Or(x.is_undef, y.is_undef, x.val + y.val > int_max, x.val + y.val < int_min), #overflow causes undef
        Or(x.is_poison, y.is_poison),
        If(x.is_undef, FreshConst("x", x.val.sort()), x.val) + If(y.is_undef, FreshConst("y", y.val.sort()), y.val),
        )

    def __div__()



class Block():
    pass
    

In [3]:
x = Const("x", SetSort(RealSort()))

z,y = Reals("z y")

# no true obviosuly
prove(Implies(And(x[z], x[y]), z == y))

def undef(x):
    return Size(x) != 1



counterexample
[x = K(Real, True), y = 3, z = 2]


In [ ]:
class State():
    ub : BoolRef
    regfile : dict[str, Var]


class Block():
    

Another model of "pure" undefinedness is to use an option type


In [4]:
MaybeReal = Datatype("Maybe<Real>")
MaybeReal.declare("none")
MaybeReal.declare("Some", ("getsome", RealSort()))
MaybeReal = MaybeReal.create()
x = Const("x", MaybeReal)
prove(x == x)

def undef(x):
    return MaybeReal.is_none(x)


proved


I've been using the theory of Real. If you have an undefined sort, there is also the option of putting the none into the reals itself. This is a kind of extended reals with a `NaN` value. Alternaitvely, you could do the /0 verion that says that dividing by zero is merely not a "standard" real / a nonstandard rwal, of which there may be many. You could think of this as multiple `NaN` values or perhaps something more exotic headed towards the hyperreals.


## Stateful Models

We can purely model state using functions that take in states and output states. The pure dependence then tracks the temporal dependence.
The state i have in mind is a env/store. A heap may be more complicated.

`State -> Option state`

`State -> list state` # non determinsim style undef

We can mix them `state -> option list state` or `state -> list option state`

Both of these can be embedded in the transition relation model. Partial functions and multisvlaue functions are both relations.
Which is quite close to modelling a tranistion relation
`State -> State -> Bool`.


We could define the values to depend on the state variable for undefined behavior like div zero. Then every "pure' function becomes impure. This seems harsh.


We might also just reject the entire idea tha purified models of stateful processes is a valid endeavor. Perhaps for some, reasoning about state is more intutitive than reasoning about more static mathematical things.
Hoare logic maybe. I don't know if you'd count this a a pure model or not. It is methematical. Predicate transformer semantics.

Writing list append iterativly vs recursviely. Familiarty  can trump other qualities of specification








# Refinement reasoning

In what sense is `x/0 -> 42` acceptable?

https://www.pypy.org/posts/2022/12/jit-bug-finding-smt-fuzzing.html
https://users.cs.utah.edu/~regehr/alive2-pldi21.pdf

gimple

refinement reasoning. How?

There's a python alive somwehe

https://github.com/AliveToolkit/alive2
https://alive2.llvm.org/ce/

In [ ]:
from z3 import *
import llvmlite
Value = Datatype("Value")
#Value.declare("undef")
Value.declare("Power", ("set", Set(BitVecSort(64))))
Value.declare("poison")
Value.declare("bv64", ("v", BitVecSort(64)))
Value = Value.create()


def plus(x,y):
    return If(Or(Value.is_undef(x), Value.is_undef(y)), Value.undef,
            If(Or(Value.is_point(x), Value.is_poison(y), Value.poison,
                  Value.bv64(Value.v(x) +  Value.v(y))))





